<a href="https://colab.research.google.com/github/TuragDev/Predicting-Crystal-System-with-ML-DL/blob/main/crystal_system_xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scikit-optimize

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 2.9 MB/s eta 0:00:00


In [2]:
!pip install xgboost

In [3]:
from google.colab import files
uploaded= files.upload()

Saving Orbital_test.csv to Orbital_test.csv
Saving Orbital_train.csv to Orbital_train.csv
Saving Sine_test.csv to Sine_test.csv
Saving Sine_train.csv to Sine_train.csv
Saving Xrd_test.csv to Xrd_test.csv
Saving Xrd_train.csv to Xrd_train.csv


In [4]:
import joblib
from sklearn.model_selection import train_test_split
import numpy as np

import pandas as pd

from sklearn.model_selection import GridSearchCV, RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.model_selection import train_test_split

import numpy as np

import pickle
import pandas as pd
from sklearn.preprocessing import LabelEncoder

import sys
# In[3]:


from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.model_selection import RepeatedStratifiedKFold
from skopt import BayesSearchCV


Sine descriptor

In [5]:
df= pd.read_csv('Sine_train.csv')
X_train=df.iloc[:,14:]
y_train=df['crystal_system']

In [6]:
df=pd.read_csv('Sine_test.csv')
X_test=df.iloc[:,14:]

y_test=df['crystal_system']

In [7]:
scaler = StandardScaler()

# fit scaler on the training data
scaler.fit(X_train)

# apply the scaler to both the training and test data
X_train_normalized = scaler.transform(X_train)

X_test_normalized = scaler.transform(X_test)
label_encoder = LabelEncoder()

# Fit and transform label encoder on target values
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.fit_transform(y_test)

In [8]:
import xgboost
from hyperopt import fmin, tpe, hp
from xgboost import XGBClassifier
from sklearn.model_selection import KFold, cross_val_score

In [9]:
# Define the search space for hyperparameters
param_space = {
    'max_depth': hp.quniform('max_depth', 3, 10, 1),
    'gamma': hp.uniform('gamma', 0, 5),
    'reg_alpha': hp.uniform('reg_alpha', 0, 5),
    'reg_lambda': hp.uniform('reg_lambda', 0, 5),
    'learning_rate': hp.loguniform('learning_rate', 0.01, 3),
    'n_estimators': hp.quniform('n_estimators', 10, 500, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.6, 1.0),
    'subsample': hp.uniform('subsample', 0.6, 1.0),
    'min_child_weight': hp.quniform('min_child_weight', 1, 10, 1)
}

# Define the objective function to minimize (negative accuracy)
def objective(params):
    model = XGBClassifier(
        max_depth=int(params['max_depth']),
        gamma=params['gamma'],
        reg_alpha=params['reg_alpha'],
        reg_lambda=params['reg_lambda'],
        learning_rate=params['learning_rate'],
        n_estimators=int(params['n_estimators']),
        min_child_weight=int(params['min_child_weight'])
    )
    model.fit(X_train_normalized, y_train_encoded)
    y_pred = model.predict(X_test_normalized)
    accuracy = accuracy_score(y_test_encoded, y_pred)
    return -accuracy  # Minimize negative accuracy

# Perform Bayesian optimization
best = fmin(
    fn=objective,
    space=param_space,
    algo=tpe.suggest,
    max_evals=50  # Number of optimization iterations
)

print("Best Hyperparameters:", best)

100%|██████████| 50/50 [00:33<00:00,  1.49trial/s, best loss: -0.738255033557047]
Best Hyperparameters: {'colsample_bytree': 0.6832609686457255, 'gamma': 2.0866642764546004, 'learning_rate': 1.6128804800182361, 'max_depth': 9.0, 'min_child_weight': 2.0, 'n_estimators': 129.0, 'reg_alpha': 2.0288073764783987, 'reg_lambda': 0.0013483446398343535, 'subsample': 0.7908974367287493}


In [10]:
import xgboost
from xgboost import XGBClassifier
from sklearn.model_selection import KFold, cross_val_score

# Define the best hyperparameters
best_hyperparameters = {
    'n_estimators': 129,
    'learning_rate': 1.6128804800182361,
    'gamma':  2.0866642764546004,
    'max_depth': 9,
    'min_child_weight': 2,
    'colsample_bytree': 0.6832609686457255,
    'subsample': 0.6246441675497391,
    'reg_alpha': 2.0288073764783987,
    'reg_lambda':0.7908974367287493
}

# Create an XGBoost classifier with the best hyperparameters
model = XGBClassifier(**best_hyperparameters)

# Perform k-fold cross-validation
cv = RepeatedKFold(n_splits=5, n_repeats=10, random_state=1)
scores = cross_val_score(model, X_train_normalized, y_train_encoded, scoring='accuracy', cv=cv, n_jobs=-1)
mean_accuracy = scores.mean()
print("Mean Accuracy:", mean_accuracy)

Mean Accuracy: 0.6573809523809524


In [11]:
model.fit(X_train_normalized, y_train_encoded)
pred = model.predict(X_train_normalized)
training_accuracy= accuracy_score(y_train_encoded,pred)
print("Accuracy on Training data: ",training_accuracy)
pred = model.predict(X_test_normalized)
test_accuracy= accuracy_score(y_test_encoded,pred)
print("Accuracy on Test data: ",test_accuracy)
Sine_Bayes_crystal = label_encoder.inverse_transform(pred)
Sine_Bayes_crystal

Accuracy on Training data:  0.8238255033557047
Accuracy on Test data:  0.6510067114093959


array(['monoclinic', 'monoclinic', 'triclinic', 'triclinic', 'monoclinic',
       'triclinic', 'monoclinic', 'triclinic', 'triclinic', 'triclinic',
       'monoclinic', 'triclinic', 'monoclinic', 'triclinic', 'triclinic',
       'monoclinic', 'monoclinic', 'monoclinic', 'monoclinic',
       'triclinic', 'triclinic', 'triclinic', 'triclinic', 'monoclinic',
       'triclinic', 'monoclinic', 'triclinic', 'monoclinic', 'monoclinic',
       'monoclinic', 'triclinic', 'triclinic', 'monoclinic', 'triclinic',
       'triclinic', 'triclinic', 'triclinic', 'triclinic', 'monoclinic',
       'triclinic', 'triclinic', 'monoclinic', 'triclinic', 'monoclinic',
       'triclinic', 'monoclinic', 'triclinic', 'monoclinic', 'triclinic',
       'triclinic', 'triclinic', 'triclinic', 'triclinic', 'monoclinic',
       'triclinic', 'triclinic', 'triclinic', 'triclinic', 'triclinic',
       'triclinic', 'monoclinic', 'triclinic', 'triclinic', 'triclinic',
       'triclinic', 'monoclinic', 'triclinic', 'tricli

OFM Descriptor

In [12]:
df= pd.read_csv('Orbital_train.csv')
X_train=df.iloc[:,14:]
y_train=df['crystal_system']

In [13]:
df=pd.read_csv('Orbital_test.csv')
X_test=df.iloc[:,14:]

y_test=df['crystal_system']

In [14]:
from sklearn.preprocessing import StandardScaler

# create a scaler object
scaler = StandardScaler()

# fit scaler on the training data
scaler.fit(X_train)

# apply the scaler to both the training and test data
X_train_normalized = scaler.transform(X_train)

X_test_normalized = scaler.transform(X_test)

In [15]:
label_encoder = LabelEncoder()

# Fit and transform label encoder on target values
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.fit_transform(y_test)

In [16]:
import xgboost
from hyperopt import fmin, tpe, hp
from xgboost import XGBClassifier
from sklearn.model_selection import KFold, cross_val_score
param_space = {
    'max_depth': hp.quniform('max_depth', 3, 10, 1),
    'gamma': hp.uniform('gamma', 0, 5),
    'reg_alpha': hp.uniform('reg_alpha', 0, 5),
    'reg_lambda': hp.uniform('reg_lambda', 0, 5),
    'learning_rate': hp.loguniform('learning_rate', 0.01, 3),
    'n_estimators': hp.quniform('n_estimators', 10, 500, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.6, 1.0),
    'subsample': hp.uniform('subsample', 0.6, 1.0),
    'min_child_weight': hp.quniform('min_child_weight', 1, 10, 1)
}

In [17]:
def objective(params):
    model = XGBClassifier(
        max_depth=int(params['max_depth']),
        gamma=params['gamma'],
        reg_alpha=params['reg_alpha'],
        reg_lambda=params['reg_lambda'],
        learning_rate=params['learning_rate'],
        n_estimators=int(params['n_estimators']),
        min_child_weight=int(params['min_child_weight'])
    )
    model.fit(X_train_normalized, y_train_encoded)
    y_pred = model.predict(X_test_normalized)
    accuracy = accuracy_score(y_test_encoded, y_pred)
    return -accuracy  # Minimize negative accuracy

# Perform Bayesian optimization
best = fmin(
    fn=objective,
    space=param_space,
    algo=tpe.suggest,
    max_evals=50  # Number of optimization iterations
)

print("Best Hyperparameters:", best)

100%|██████████| 50/50 [01:52<00:00,  2.25s/trial, best loss: -0.6778523489932886]
Best Hyperparameters: {'colsample_bytree': 0.7835314983643119, 'gamma': 2.5936523613664835, 'learning_rate': 1.177417006484881, 'max_depth': 5.0, 'min_child_weight': 3.0, 'n_estimators': 464.0, 'reg_alpha': 3.1924020642742175, 'reg_lambda': 4.20813042059598, 'subsample': 0.7726017420235414}


In [18]:
import xgboost
from xgboost import XGBClassifier
from sklearn.model_selection import KFold, cross_val_score

# Define the best hyperparameters
best_hyperparameters = {
    'n_estimators': 464,
    'learning_rate': 1.177417006484881,
    'gamma':  2.5936523613664835,
    'max_depth': 5,
    'min_child_weight': 3,
    'colsample_bytree':  0.7835314983643119,
    'subsample':0.7726017420235414,
    'reg_alpha': 3.1924020642742175,
    'reg_lambda':4.20813042059598
}



In [19]:
model = XGBClassifier(**best_hyperparameters)

# Perform k-fold cross-validation
cv = RepeatedKFold(n_splits=5, n_repeats=10, random_state=1)
scores = cross_val_score(model, X_train_normalized, y_train_encoded, scoring='accuracy', cv=cv, n_jobs=-1)
mean_accuracy = scores.mean()
print("Mean Accuracy:", mean_accuracy)
model.fit(X_train_normalized, y_train_encoded)
pred = model.predict(X_train_normalized)
training_accuracy= accuracy_score(y_train_encoded,pred)
print("Accuracy on Training data: ",training_accuracy)
pred = model.predict(X_test_normalized)
test_accuracy= accuracy_score(y_test_encoded,pred)
print("Accuracy on Test data: ",test_accuracy)
Orbital_Bayes_crystal = label_encoder.inverse_transform(pred)
Orbital_Bayes_crystal

Mean Accuracy: 0.6391596638655462
Accuracy on Training data:  0.6756302521008404
Accuracy on Test data:  0.6442953020134228


array(['cubic', 'cubic', 'triclinic', 'triclinic', 'triclinic',
       'triclinic', 'triclinic', 'triclinic', 'triclinic', 'triclinic',
       'triclinic', 'triclinic', 'triclinic', 'triclinic', 'triclinic',
       'triclinic', 'triclinic', 'monoclinic', 'triclinic', 'triclinic',
       'triclinic', 'triclinic', 'monoclinic', 'triclinic', 'triclinic',
       'monoclinic', 'triclinic', 'monoclinic', 'triclinic', 'triclinic',
       'triclinic', 'triclinic', 'triclinic', 'triclinic', 'triclinic',
       'triclinic', 'triclinic', 'triclinic', 'triclinic', 'triclinic',
       'triclinic', 'monoclinic', 'triclinic', 'triclinic', 'triclinic',
       'triclinic', 'triclinic', 'triclinic', 'triclinic', 'triclinic',
       'triclinic', 'triclinic', 'triclinic', 'triclinic', 'triclinic',
       'triclinic', 'triclinic', 'triclinic', 'triclinic', 'triclinic',
       'triclinic', 'triclinic', 'triclinic', 'triclinic', 'triclinic',
       'triclinic', 'triclinic', 'triclinic', 'triclinic', 'triclin

Xrd

In [33]:
df= pd.read_csv('Xrd_train.csv')
X_train=df.iloc[:,14:]
y_train=df['crystal_system']

In [34]:
df=pd.read_csv('Xrd_test.csv')
X_test=df.iloc[:,14:]

y_test=df['crystal_system']

In [35]:
scaler = StandardScaler()

# fit scaler on the training data
scaler.fit(X_train)

# apply the scaler to both the training and test data
X_train_normalized = scaler.transform(X_train)

X_test_normalized = scaler.transform(X_test)
label_encoder = LabelEncoder()

# Fit and transform label encoder on target values
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.fit_transform(y_test)

In [36]:
import xgboost
from hyperopt import fmin, tpe, hp
from xgboost import XGBClassifier
from sklearn.model_selection import KFold, cross_val_score
param_space = {
    'max_depth': hp.quniform('max_depth', 3, 10, 1),
    'gamma': hp.uniform('gamma', 0, 5),
    'reg_alpha': hp.uniform('reg_alpha', 0, 5),
    'reg_lambda': hp.uniform('reg_lambda', 0, 5),
    'learning_rate': hp.loguniform('learning_rate', 0.01, 3),
    'n_estimators': hp.quniform('n_estimators', 10, 500, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.6, 1.0),
    'subsample': hp.uniform('subsample', 0.6, 1.0),
    'min_child_weight': hp.quniform('min_child_weight', 1, 10, 1)
}

In [37]:
def objective(params):
    model = XGBClassifier(
        max_depth=int(params['max_depth']),
        gamma=params['gamma'],
        reg_alpha=params['reg_alpha'],
        reg_lambda=params['reg_lambda'],
        learning_rate=params['learning_rate'],
        n_estimators=int(params['n_estimators']),
        min_child_weight=int(params['min_child_weight'])
    )
    model.fit(X_train_normalized, y_train_encoded)
    y_pred = model.predict(X_test_normalized)
    accuracy = accuracy_score(y_test_encoded, y_pred)
    return -accuracy  # Minimize negative accuracy

# Perform Bayesian optimization
best = fmin(
    fn=objective,
    space=param_space,
    algo=tpe.suggest,
    max_evals=50  # Number of optimization iterations
)

print("Best Hyperparameters:", best)

100%|██████████| 50/50 [01:33<00:00,  1.87s/trial, best loss: -0.6912751677852349]
Best Hyperparameters: {'colsample_bytree': 0.9686622065631607, 'gamma': 0.023697090147251076, 'learning_rate': 2.0247860840671867, 'max_depth': 7.0, 'min_child_weight': 4.0, 'n_estimators': 224.0, 'reg_alpha': 1.9013490884575868, 'reg_lambda': 3.613352738350256, 'subsample': 0.9217785928923488}


In [44]:
import xgboost
from xgboost import XGBClassifier
from sklearn.model_selection import KFold, cross_val_score

# Define the best hyperparameters
best_hyperparameters = {
    'n_estimators': 224,
    'learning_rate': 2.0247860840671867,
    'gamma':  0.023697090147251076,
    'max_depth': 7,
    'min_child_weight': 4,
    'colsample_bytree':  0.9686622065631607,
    'subsample': 0.9217785928923488,
    'reg_alpha': 1.9013490884575868,
    'reg_lambda': 3.613352738350256
}


In [45]:
model = XGBClassifier(**best_hyperparameters)

# Perform k-fold cross-validation
cv = RepeatedKFold(n_splits=5, n_repeats=10, random_state=1)
scores = cross_val_score(model, X_train_normalized, y_train_encoded, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
mean_accuracy = scores.mean()
print("Mean Accuracy:", mean_accuracy)
model.fit(X_train_normalized, y_train_encoded)
pred = model.predict(X_train_normalized)
training_accuracy= accuracy_score(y_train_encoded,pred)
print("Accuracy on Training data: ",training_accuracy)
pred = model.predict(X_test_normalized)
test_accuracy= accuracy_score(y_test_encoded,pred)
print("Accuracy on Test data: ",test_accuracy)
Xrd_Bayes_crystal = label_encoder.inverse_transform(pred)
Xrd_Bayes_crystal

Mean Accuracy: 0.6814019607843136
Accuracy on Training data:  0.9983221476510067
Accuracy on Test data:  0.6375838926174496


array(['monoclinic', 'tetragonal', 'monoclinic', 'triclinic', 'triclinic',
       'monoclinic', 'triclinic', 'triclinic', 'triclinic', 'monoclinic',
       'monoclinic', 'triclinic', 'monoclinic', 'triclinic', 'monoclinic',
       'monoclinic', 'triclinic', 'triclinic', 'monoclinic', 'triclinic',
       'triclinic', 'monoclinic', 'triclinic', 'monoclinic', 'triclinic',
       'triclinic', 'monoclinic', 'trigonal', 'monoclinic', 'monoclinic',
       'triclinic', 'triclinic', 'triclinic', 'triclinic', 'triclinic',
       'triclinic', 'triclinic', 'triclinic', 'triclinic', 'monoclinic',
       'triclinic', 'monoclinic', 'triclinic', 'triclinic', 'triclinic',
       'monoclinic', 'triclinic', 'triclinic', 'triclinic', 'triclinic',
       'triclinic', 'triclinic', 'triclinic', 'triclinic', 'triclinic',
       'triclinic', 'triclinic', 'monoclinic', 'monoclinic',
       'orthorhombic', 'triclinic', 'monoclinic', 'triclinic',
       'triclinic', 'triclinic', 'monoclinic', 'triclinic', 'tricli